In [1]:
# pip install torch torchvision
# pip install torch torchvision torchaudio

In [1]:
import os
import numpy as np

PATH = '../data/data_preprocessing/v0.8/augmented'
images, labels = [], []

for dirname, _, filenames in os.walk(PATH):
    for filename in filenames:
        images.append(os.path.join(dirname, filename).split('/')[-1])  # /folder/blabla/xxx.jpg
        labels.append(os.path.basename(dirname))  # Menggunakan nama folder sebagai label

# Menggunakan np.unique untuk mendapatkan kelas unik
# label2cat = np.unique(labels)
# print("Kelas :", label2cat)

# Membuat kamus untuk memetakan label ke kelas numerik
# label_to_numeric = {label: idx for idx, label in enumerate(label2cat)}
# print("Kamus Label ke Numerik:", label_to_numeric)
# label_to_numeric

In [3]:
import pandas as pd

# Membuat DataFrame dari daftar gambar dan label
datas = pd.DataFrame({'image_id': images, 'label': labels})

# Menampilkan lima baris pertama dan bentuk DataFrame
print(datas.head())
print("Bentuk DataFrame:", datas.shape)


                                    image_id label
0  augmented\ba\ba116.pred3.b_aug_0_1033.png    ba
1  augmented\ba\ba116.pred3.b_aug_0_1069.png    ba
2  augmented\ba\ba116.pred3.b_aug_0_1077.png    ba
3  augmented\ba\ba116.pred3.b_aug_0_1095.png    ba
4  augmented\ba\ba116.pred3.b_aug_0_1097.png    ba
Bentuk DataFrame: (9860, 2)


In [4]:
from sklearn.model_selection import train_test_split

# # Memisahkan data menjadi data latih (train) dan data uji (test)
# X_train, X_test, y_train, y_test = train_test_split(
#     datas['image_id'].values, datas['label'].values, 
#     test_size=0.3, stratify=datas['label'].values, 
#     random_state=42)

# # Menampilkan jumlah data latih dan data uji
# print("Jumlah data latih:", len(X_train))
# print("Jumlah data uji:", len(X_test))
# # len(X_train), len(X_test)

X_train, X_val, y_train, y_val = train_test_split(
    datas['image_id'].values, datas['label'].values, 
    test_size=0.3, stratify=datas['label'].values, 
    random_state=24)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,
    test_size=0.5, stratify=y_val, 
    random_state=24)


len(X_train), len(X_val), len(X_test)

(6902, 1479, 1479)

In [5]:
from torch.utils.data import Dataset, DataLoader

class customDataset(Dataset):
    def __init__(self, x, y, path, maps_label=label_to_numeric, transform=None):
        self.X = x
        self.y = y
        self.path = path
        self.label_to_numeric = maps_label
        self.transform = transform
        
    def __getitem__(self, idx):
        # img = Image.open(self.path + str(self.y[idx]) + '/' + str(self.X[idx])).convert('RGB')
        img = Image.open(self.path + str(self.y[idx]) + '/' + str(self.X[idx]))
        img = img.convert('L')  # Mengonversi gambar ke citra grayscale
        label = self.y[idx]
        label = self.label_2_ints(label)
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, label
    
    def label_2_ints(self, x):
        label_id = None
        for key, values in self.label_to_numeric.items():
            if x == key:
                label_id = values
        return label_id
    
    def __len__(self):
        return len(self.X)

In [ ]:
import torchvision.transforms as transforms

crop_size = 224
bs = 64
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(crop_size, scale=(0.9, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

val_transform = transforms.Compose([
    transforms.Resize(230),
    transforms.CenterCrop(crop_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
train_set = customDataset(X_train, y_train, PATH, transform=train_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=2)

val_set = customDataset(X_val, y_val, PATH, transform=val_transform)
valloader = DataLoader(val_set, batch_size=bs, shuffle=True)

test_set = customDataset(X_test, y_test, PATH, transform=val_transform)
testloader = DataLoader(test_set, shuffle=True)

In [ ]:
feature, target = next(iter(trainloader))
feature.shape, len(trainloader)

In [ ]:
fig, axes = plt.subplots(2,5, figsize=(15, 7))
for img, label, ax in zip(feature, target, axes.flatten()):
  ax.imshow(img.permute(1,2,0).cpu())
  font = {"color":'g'}
  label = label2cat[label.item()]
  ax.set_title(f"Label: {label}", fontdict=font);
  ax.axis("off");

In [ ]:
class CustomMobileNetv2(nn.Module):
  def __init__(self, output_size):
    super().__init__()
    self.mnet = mobilenet_v2(pretrained=True)
    self.freeze()

    self.mnet.classifier = nn.Sequential(
        nn.Linear(1280, 256),
        nn.ReLU(),
        nn.Linear(256, output_size),
        nn.LogSoftmax(1)
    )

  def forward(self, x):
    return self.mnet(x)
  
  def freeze(self):
    for param in self.mnet.parameters():
      param.requires_grad = False

  def unfreeze(self):
    for param in self.mnet.parameters():
      param.requires_grad = True

In [ ]:
config = set_config({
    'batch_size': bs,
    'crop_size': crop_size,
    'output_size': len(label2cat)
})

In [ ]:
model = CustomMobileNetv2(config.output_size).to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
callback = Callback(model, config, early_stop_patience=2, outdir='model')

In [ ]:
def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
  if mode == 'train':
    model.train()
  elif mode == 'val':
    model.eval()
  
  cost = correct = 0
  for feature, target in dataloader:
    feature, target = feature.to(device), target.to(device)
    output = model(feature)
    loss = criterion(output, target)

    if mode == 'train':
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    
    cost += loss.item() * feature.shape[0]
    correct += (output.argmax(1) == target).sum().item()
  cost = cost/len(dataset)
  acc = correct/len(dataset)
  return cost, acc

In [ ]:
while True:
  train_cost, train_score = loop_fn('train', train_set, trainloader, model, criterion, optimizer, device)
  with torch.no_grad():
    test_cost, test_score = loop_fn('val', val_set, valloader, model, criterion, optimizer, device)

  # Logging
  callback.log(train_cost, test_cost, train_score, test_score)

  # Checkpoint
  callback.save_checkpoint()

  # Runtime Plotting
  callback.cost_runtime_plotting()
  callback.score_runtime_plotting()

  # Early Stopping
  if callback.early_stopping(model, monitor='test_score'):
    callback.plot_cost()
    callback.plot_score()
    break

In [ ]:
model.unfreeze()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

callback.reset_early_stop()
callback.early_stop_patience = 6

In [ ]:
while True:
  train_cost, train_score = loop_fn('train', train_set, trainloader, model, criterion, optimizer, device)
  with torch.no_grad():
    test_cost, test_score = loop_fn('val', val_set, valloader, model, criterion, optimizer, device)

  # Logging
  callback.log(train_cost, test_cost, train_score, test_score)

  # Checkpoint
  callback.save_checkpoint()

  # Runtime Plotting
  callback.cost_runtime_plotting()
  callback.score_runtime_plotting()

  # Early Stopping
  if callback.early_stopping(model, monitor='test_score'):
    callback.plot_cost()
    callback.plot_score()
    break

In [ ]:
for feature, target in valloader:
  feature, target = feature.to(device), target.to(device)
  with torch.no_grad():
    model.eval()
    output = model(feature)
    preds = output.argmax(1)

fig, axes = plt.subplots(6, 6, figsize=(24, 24))
for img, label, pred, ax in zip(feature, target, preds, axes.flatten()):
  ax.imshow(img.permute(1,2,0).cpu())
  font = {"color":'r'} if label != pred else {"color": 'g'}
  label, pred = label2cat[label.item()], label2cat[pred.item()]
  ax.set_title(f"Label: {label}\nPred: {pred}", fontdict=font);
  ax.axis("off");

In [ ]:
accuracy = []
for feature, target in testloader:
  feature, target = feature.to(device), target.to(device)
  with torch.no_grad():
    model.eval()
    output = model(feature)
    preds = output.argmax(1)
    if(target == preds):
      acc = 1
    else: acc = 0
    accuracy.append(acc)
accuracy = np.array(accuracy)
accuracy.mean()

In [ ]:
# class Jawo(Dataset):
#     def __init__(self, x, y, path, maps_label=dicts, transform=None):
#         self.X = x
#         self.y = y
#         self.path = path
#         self.dicts = maps_label
#         self.transform = transform
        
#     def __getitem__(self, idx):
#         img = Image.open(self.path + str(self.y[idx]) + '/' + str(self.X[idx])).convert('RGB')
#         label = self.y[idx]
#         label = self.label_2_ints(label)
        
#         if self.transform is not None:
#             img = self.transform(img)
            
#         return img, label
    
#     def label_2_ints(self, x):
#         label_id = None
#         for key, values in self.dicts.items():
#             if x == key:
#                 label_id = values
#         return label_id
    
#     def __len__(self):
#         return len(self.X)

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(
#     datas['image_id'].values, datas['label'].values, 
#     test_size=0.3, stratify=datas['label'].values, 
#     random_state=24)

# X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,
#     test_size=0.5, stratify=y_val, 
#     random_state=24)


# len(X_train), len(X_val), len(X_test)

In [ ]:
# class customDataset(Dataset):
#     def __init__(self, x, y, path, maps_label=dicts, transform=None):
#         self.X = x
#         self.y = y
#         self.path = path
#         self.dicts = maps_label
#         self.transform = transform
        
#     def __getitem__(self, idx):
#         img = Image.open(self.path + str(self.y[idx]) + '/' + str(self.X[idx])).convert('RGB')
#         label = self.y[idx]
#         label = self.label_2_ints(label)
        
#         if self.transform is not None:
#             img = self.transform(img)
            
#         return img, label
    
#     def label_2_ints(self, x):
#         label_id = None
#         for key, values in self.dicts.items():
#             if x == key:
#                 label_id = values
#         return label_id
    
#     def __len__(self):
#         return len(self.X)